# Limpieza de datos: Contenido añadido a Netflix entre 2008 y 2021
> Dataset disponible en https://www.kaggle.com/datasets/ariyoomotade/netflix-data-cleaning-analysis-and-visualization

El objetivo de este dataset es, principalmente, la limpieza y visualización de datos. Manos a la obra.

In [40]:
import pandas as pd
df = pd.read_csv("./datasets/netflix1.csv")

df.head()

,show_id,type,title,director,country,date_added,release_year,rating,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,9/25/2021,2020,PG-13,90 min,Documentaries
1,s3,TV Show,Ganglands,Julien Leclercq,France,9/24/2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act..."
2,s6,TV Show,Midnight Mass,Mike Flanagan,United States,9/24/2021,2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries"
3,s14,Movie,Confessions of an Invisible Girl,Bruno Garotti,Brazil,9/22/2021,2021,TV-PG,91 min,"Children & Family Movies, Comedies"
4,s8,Movie,Sankofa,Haile Gerima,United States,9/24/2021,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies"


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8790 entries, 0 to 8789
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8790 non-null   object
 1   type          8790 non-null   object
 2   title         8790 non-null   object
 3   director      8790 non-null   object
 4   country       8790 non-null   object
 5   date_added    8790 non-null   object
 6   release_year  8790 non-null   int64 
 7   rating        8790 non-null   object
 8   duration      8790 non-null   object
 9   listed_in     8790 non-null   object
dtypes: int64(1), object(9)
memory usage: 686.8+ KB


Generalmente, se consideran 6 dimensiones bajo la cual evaluar la calidad de datos:
1. Completitud (*completeness*): Los registros están 100 completos, ¿o falta información?
2. Consistencia (*consistency*): En todos los casos, ¿los datos representan la misma información de la misma manera?
3. Unicidad (*uniqueness*): En todos los casos, no existen múltiples registros de la misma información.
4. Validez (*validity*): Los datos que conforman una información están dentro de un dominio definido de valores.
5. Exactitud (*accuracy*): ¿Los datos describen correctamente la información?
6. Usabilidad (*usability*): ¿Los datos son relevantes, accesibles, y mantenibles con el nivel deseado de precisión?

(tomado de https://www.kansascityfed.org/Technical%20Briefings/documents/4986/tb-Discussing%20Data:%20Evaluating%20Data%20Quality.pdf)

En este caso, me limitaré a evaluar y corregir hasta un grado aceptable las primeras 4 dimensiones, comenzando por la primera, la cual dio un resultado positivo: El 100% de las filas están completas, al menos en una primera instancia. A continuación, se verificará más a profundidad si esto se cumple o no.

In [42]:
import ydata_profiling as yp
profile = yp.ProfileReport(df, title="Netflix Data Cleaning Report", explorative=True)
profile.to_file("netflix_data_cleaning_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 74.00it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Del reporte surgen algunas cosas a notar:
- En las siguientes características se evidencian valores marcados como `not given`: 
    - `director`
    - `country`
- La característica `duration` tiene tanto valores numéricos como categóricos. Adicionalmente, las palabras más repetidas son `seasons`, `season`, y `min`.

In [43]:
df.groupby("director").size().sort_values(ascending=False)

director
Not Given                                2588
Rajiv Chilaka                              20
Alastair Fothergill                        18
Raúl Campos, Jan Suter                     18
Suhas Kadav                                16
                                         ... 
Jose Gomez                                  1
Jose Javier Reyes                           1
Billy Lyons                                 1
Joseduardo Giordano, Sergio Goyri Jr.       1
Khalid Mohamed                              1
Length: 4528, dtype: int64

In [44]:
df.groupby("country").size().sort_values(ascending=False)

country
United States     3240
India             1057
United Kingdom     638
Pakistan           421
Not Given          287
                  ... 
Soviet Union         1
Paraguay             1
Puerto Rico          1
Somalia              1
Zimbabwe             1
Length: 86, dtype: int64

In [45]:
df['duration_type'] = df['duration'].map(lambda duration: duration.split(" ").pop())
df['duration_value'] = df['duration'].map(lambda duration: int(duration.split(" ")[0]))

df.head()

,show_id,type,title,director,country,date_added,release_year,rating,duration,listed_in,duration_type,duration_value
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,9/25/2021,2020,PG-13,90 min,Documentaries,min,90
1,s3,TV Show,Ganglands,Julien Leclercq,France,9/24/2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",Season,1
2,s6,TV Show,Midnight Mass,Mike Flanagan,United States,9/24/2021,2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",Season,1
3,s14,Movie,Confessions of an Invisible Girl,Bruno Garotti,Brazil,9/22/2021,2021,TV-PG,91 min,"Children & Family Movies, Comedies",min,91
4,s8,Movie,Sankofa,Haile Gerima,United States,9/24/2021,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies",min,125


In [46]:
df['duration_type'].value_counts()

duration_type
min        6126
Season     1791
Seasons     873
Name: count, dtype: int64